In [35]:
# 이미지 데이터 전처리 # 데이터 증식 사용
import keras
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=90,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.6,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255) # 검증데이터는 증식하지 않는다.

train_generator = train_datagen.flow_from_directory(
    '/Users/kibeomkim/Desktop/openlogo_split/train/', 
    classes=['other','samsung'],
    target_size=(150, 150),
    batch_size=100, 
    class_mode='binary', 
)

validation_generator = test_datagen.flow_from_directory(
    '/Users/kibeomkim/Desktop/openlogo_split/test/',
    classes=['other', 'samsung'],
    target_size=(150,150),
    batch_size=100,
    class_mode='binary', 
)

Found 18752 images belonging to 2 classes.
Found 8331 images belonging to 2 classes.


In [36]:
from keras import models 
from keras import layers 

# VGG16 합성곱 기반 층 만들기 

from keras.applications import vgg16
from keras import optimizers

# 합성곱 기반 층
conv_base = vgg16.VGG16(
    weights='imagenet',
    include_top = False,
    input_shape=(150, 150, 3)
)

# 모델 정의
model = models.Sequential() 
model.add(conv_base)# 합성곱 기반 층
model.add(layers.Flatten()) # 합성곱 기반 층 출력 1차원 텐서(벡터로)변형
model.add(layers.BatchNormalization())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(1, activation='sigmoid'))

# 합성곱 기반 층 가중치 동결 
conv_base.trainable = False
print(f'{len(model.trainable_weights)}') 


# 모델 컴파일 
model.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.adam_v2.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

# 모델 훈련 
history = model.fit_generator(
    train_generator,
    steps_per_epoch = 187, 
    epochs=10,
    validation_data = validation_generator, 
    validation_steps=83)

8
Epoch 1/10


2022-02-25 11:22:42.235534: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


187/187 [==============================] - ETA: 0s - loss: 0.5629 - accuracy: 0.7393

2022-02-25 11:25:53.030620: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


187/187 [==============================] - 257s 1s/step - loss: 0.5629 - accuracy: 0.7393 - val_loss: 0.2087 - val_accuracy: 0.9977
Epoch 2/10
187/187 [==============================] - 262s 1s/step - loss: 0.1435 - accuracy: 0.9773 - val_loss: 0.0378 - val_accuracy: 0.9977
Epoch 3/10
187/187 [==============================] - 255s 1s/step - loss: 0.0439 - accuracy: 0.9967 - val_loss: 0.0205 - val_accuracy: 0.9977
Epoch 4/10
187/187 [==============================] - 274s 1s/step - loss: 0.0266 - accuracy: 0.9974 - val_loss: 0.0173 - val_accuracy: 0.9977
Epoch 5/10
187/187 [==============================] - 271s 1s/step - loss: 0.0227 - accuracy: 0.9975 - val_loss: 0.0155 - val_accuracy: 0.9977
Epoch 6/10
187/187 [==============================] - 258s 1s/step - loss: 0.0195 - accuracy: 0.9975 - val_loss: 0.0152 - val_accuracy: 0.9977
Epoch 7/10
149/187 [======================>.......] - ETA: 37s - loss: 0.0181 - accuracy: 0.9976

KeyboardInterrupt: 

In [14]:
model.save('/Users/kibeomkim/Desktop/models_saved/samsung.h5') # 모델 저장 

In [44]:
from keras.preprocessing import image 
from keras.applications.vgg16 import preprocess_input

img_path = '/Users/kibeomkim/Desktop/sh.png'

img = image.load_img(img_path, target_size=(150,150))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

x = preprocess_input(x)

result = model.predict(x) ; print(result)

[[1.]]


In [46]:
import os 
valid_list = os.listdir('/Users/kibeomkim/Desktop/openlogo_split/test/samsung') 

def galaxy(name) : 
    img = image.load_img(f'/Users/kibeomkim/Desktop/openlogo_split/test/samsung/{name}', target_size=(150,150))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return model.predict(x) 

result = [galaxy(image) for image in valid_list]

In [47]:
print(len(result))
print(len([x for x in result if x > 0.5]))

19
12


In [48]:
result

[array([[0.99999785]], dtype=float32),
 array([[0.]], dtype=float32),
 array([[1.]], dtype=float32),
 array([[0.]], dtype=float32),
 array([[0.04159725]], dtype=float32),
 array([[7.6443085e-10]], dtype=float32),
 array([[1.]], dtype=float32),
 array([[7.630768e-09]], dtype=float32),
 array([[1.]], dtype=float32),
 array([[1.]], dtype=float32),
 array([[1.]], dtype=float32),
 array([[0.]], dtype=float32),
 array([[0.99999857]], dtype=float32),
 array([[0.99999726]], dtype=float32),
 array([[1.]], dtype=float32),
 array([[1.]], dtype=float32),
 array([[0.001118]], dtype=float32),
 array([[0.9999598]], dtype=float32),
 array([[1.]], dtype=float32)]

In [49]:
# 인스타그램 이미지 투입해보자. 

valid_list = os.listdir('/Users/kibeomkim/Desktop/인스타그램_크롤링_결과/#galaxys21_2000') 

def galaxy(name) : 
    img = image.load_img(f'/Users/kibeomkim/Desktop/인스타그램_크롤링_결과/#galaxys21_2000/{name}', target_size=(150,150))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return model.predict(x) 

result = [galaxy(image) for image in valid_list]

In [50]:
len([x for x in result if x > 0.5]) 

1066

In [52]:
1066/len(result)

0.45536095685604444

# 모델이 일반화 성능 지표는 높게 나오는데, 삼성과 전혀 관련 없는 이미지들까지 '삼성'으로 분류해버린다. 
# 예컨대 훈련 이미지 중 갤럭시 스마트폰의 바bar 형태를 삼성의 특징이라 생각하고 분류해 내는 듯 하다. 
# 실제로 사각형 이미지를 넣었을 때 '삼성' 이라고 분류했다. 
# 합성곱 신경망이 어떻게 동작하는 지 이론적 근거를 좀 더 자세히 공부하고, 다시 모델 세우자. 